In [ ]:
# Observational samples (binary; 0/1). Length = 60.
Z = np.array([0,0,0,1,1,1,1,1,0,1,0,1,1,1,0,0,0,0,1,0,0,0,1,0,0,1,0,1,1,0,1,1,0,0,0,0,1,1,0,0,1,0,1,1,0,1,0,0,1,1,0,0,1,0,0,1,1,0], dtype=int)
X = np.array([0,0,0,1,1,0,1,0,0,0,1,1,1,0,0,0,0,0,0,0,1,0,1,0,0,1,0,1,1,0,1,1,0,0,0,0,1,1,0,0,0,0,1,1,0,1,1,0,0,1,0,0,1,0,0,1,0,0], dtype=int)
M = np.array([1,0,0,1,1,1,1,1,1,1,0,1,1,0,0,0,0,0,1,0,0,0,1,0,0,1,0,1,1,0,1,1,0,0,0,0,1,1,0,0,1,0,1,1,0,1,0,0,1,1,0,0,1,0,0,1,1,0], dtype=int)
Y = np.array([0,0,0,1,0,1,1,1,1,1,0,0,1,0,0,0,0,0,1,0,1,0,1,0,0,1,0,1,1,0,1,1,0,0,0,0,1,0,0,0,1,0,1,1,0,1,0,0,0,1,0,0,1,0,0,1,0,0], dtype=int)

# Interventional/experimental results you can feed as constraints (chosen completly aribitrary)
# p_doZ[z] = P(Y=1 | do(Z=1))
p_doZ = 0.568

# p_doM[m] = P(Y=1 | do(M=1))
p_doM = 0.8

lb, ub = bounds_PY_doX1(Z, X, M, Y, p_doM=p_doM, p_doZ=p_doZ)  # purely observational
print(ub-lb)
#none: 0.6206896529999999
#p_doM: 0.6206896580000001 (wider: The experiment result is not compatible with the observational data we had -> likely model misspecification)
#p_doZ: 0.4837241339999999
#both:  0.48372413400000003


0.48372413400000003


In [1]:
import numpy as np

# ---------- deterministic SCM (no noise in structural functions) ----------
def gen_observational_data(n=100_000, seed=213, pW=0.3, pU=0.6):
    rng = np.random.default_rng(seed)
    W = (rng.random(n) < pW).astype(int)
    U = (rng.random(n) < pU).astype(int)

    # Structural equations (deterministic given parents)
    Z = W
    X = Z * (1 - U)            # X = Z AND (NOT U)
    M = X
    Y = ((M & W) | U).astype(int)  # Y = (M AND W) OR U

    return Z, X, M, Y, W, U, pW, pU

def true_probs_from_scm(pW, pU):
    # Ground truth target
    p_doX1 = 1 - (1 - pW) * (1 - pU)       # P(W=1 or U=1)
    # Experiments
    p_doZ1 = pU + (1 - pU) * pW            # = P(U=1) + P(U=0)*P(W=1)
    p_doM1 = 1 - (1 - pW) * (1 - pU)       # same as do(X=1) in this SCM
    return p_doX1, p_doZ1, p_doM1


# ---------- demo runner (assumes bounds_PY_doX1 is already defined) ----------
def demo_bounds(seed=123):
    from brute_force import bounds_PY_doX1
    # Generate large-N observational data to kill sampling noise
    Z, X, M, Y, W, U, pW, pU = gen_observational_data(n=200_000, seed=seed, pW=0.3, pU=0.6)

    # Ground-truth values from SCM
    p_doX1_true, p_doZ1_true, p_doM1_true = true_probs_from_scm(pW, pU)

    # 1) Observational only
    lb_obs, ub_obs = bounds_PY_doX1(Z, X, M, Y)
    # 2) With do(Z=1)
    lb_Z, ub_Z = bounds_PY_doX1(Z, X, M, Y, p_doZ=p_doZ1_true)
    # 3) With do(M=1)
    lb_M, ub_M = bounds_PY_doX1(Z, X, M, Y, p_doM=p_doM1_true)
    # 4) With both
    lb_b, ub_b = bounds_PY_doX1(Z, X, M, Y, p_doZ=p_doZ1_true, p_doM=p_doM1_true)

    def show(tag, lb, ub):
        print(f"{tag:12s}  [{lb:.6f}, {ub:.6f}]  width={ub-lb:.6f}  "
              f"(reduction to observational: {(ub_obs - lb_obs) - (ub - lb):.6f})")

    print("SCM ground truth:")
    print(f"  P(Y=1 | do(X=1)) = {p_doX1_true:.6f}")
    print(f"  p_(Z=1)          = {p_doZ1_true:.6f}")
    print(f"  p_(M=1)          = {p_doM1_true:.6f}")
    print()
    show("observational", lb_obs, ub_obs)
    show("+ do(Z=1)",     lb_Z,   ub_Z)
    show("+ do(M=1)",     lb_M,   ub_M)
    show("+ both",        lb_b,   ub_b)


# --- run the demo ---

demo_bounds(12)


SCM ground truth:
  P(Y=1 | do(X=1)) = 0.720000
  p_(Z=1)          = 0.720000
  p_(M=1)          = 0.720000

observational  [0.119925, 1.000000]  width=0.880075  (reduction to observational: 0.000000)
+ do(Z=1)     [0.120030, 1.000000]  width=0.879970  (reduction to observational: 0.000105)
+ do(M=1)     [0.119925, 1.000000]  width=0.880075  (reduction to observational: 0.000000)
+ both        [0.120030, 1.000000]  width=0.879970  (reduction to observational: 0.000105)


In [2]:
import importlib
import symbolic_solver
importlib.reload(symbolic_solver)
from symbolic_solver import parametric_bounds_PY_doX1
from brute_force import bounds_PY_doX1
from symbolic_solver import evaluate_bound_piecewise

# Generate large-N observational data to kill sampling noise
Z, X, M, Y, W, U, pW, pU = gen_observational_data(n=200_000, seed=12, pW=0.3, pU=0.6)

# Ground-truth values from SCM
p_doX1_true, p_doZ1_true, p_doM1_true = true_probs_from_scm(pW, pU)


# 1) Only do(Z)
resZ = parametric_bounds_PY_doX1(Z, X, M, Y, use_doZ=True, use_doM=False)
print("Upper pieces f(p_doZ, p_doM):")
for s in resZ["upper"]["strings"]: print("  ", s)
print("Lower pieces g_Z(p_doZ, p_doM):")
for s in resZ["lower"]["strings"]: print("  ", s)


# lb, ub = bounds_PY_doX1(Z, X, M, Y, p_doZ=0.7)  # true LP bounds
# print(f"True LP Bounds: [{lb:.6f}, {ub:.6f}]  width={ub-lb:.6f}")

resZ

Upper pieces f(p_doZ, p_doM):
   0.2801050030 + 1.0000000000*p_doZ
   1.0000000020 + -0.0000000000*p_doZ
Lower pieces g_Z(p_doZ, p_doM):
   0.1199250000 + 0.0000000000*p_doZ
   -0.5999700000 + 1.0000000000*p_doZ


{'upper': {'pieces': [(1.0, 0.0, 0.280105003), (-0.0, 0.0, 1.000000002)],
  'strings': ['0.2801050030 + 1.0000000000*p_doZ',
   '1.0000000020 + -0.0000000000*p_doZ']},
 'lower': {'pieces': [(0.0, 0.0, 0.119925), (1.0, 0.0, -0.59997)],
  'strings': ['0.1199250000 + 0.0000000000*p_doZ',
   '-0.5999700000 + 1.0000000000*p_doZ']},
 'note': 'Bounds are envelopes (upper = max of pieces, lower = min of pieces).'}

In [ ]:
#Goal: return the best case bounds for a given intervention

